In [1]:
import os
import numpy as np
import trimesh
from scipy.ndimage import zoom
from tqdm import tqdm
import kagglehub

In [2]:
# Download latest version
path = kagglehub.dataset_download("balraj98/modelnet10-princeton-3d-object-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/rezasarkhosh/.cache/kagglehub/datasets/balraj98/modelnet10-princeton-3d-object-dataset/versions/1


In [3]:
dataset_root = os.path.join(path, "ModelNet10") 
selected_classes = ['chair', 'table', 'sofa', 'monitor', 'desk', 'bed', 'bathtub', 'night_stand', 'toilet', 'dresser']
voxel_dim = 32  
num_orientations = 12  

In [4]:
def rotate_mesh(mesh, angle_deg):
    angle_rad = np.deg2rad(angle_deg)
    rotation_matrix = trimesh.transformations.rotation_matrix(
        angle_rad, direction=[0, 0, 1], point=[0, 0, 0]
    )
    mesh_rotated = mesh.copy()
    mesh_rotated.apply_transform(rotation_matrix)
    return mesh_rotated

In [5]:
def load_voxel_dataset_with_orientation(dataset_path, voxel_dim=32, selected_classes=None):
    X, y_class, y_orientation = [], [], []

    if selected_classes is None:
        selected_classes = sorted(os.listdir(dataset_path))
    
    label_map = {cls_name: idx for idx, cls_name in enumerate(selected_classes)}

    for cls_name in tqdm(selected_classes, desc="Classes"):
        class_path = os.path.join(dataset_path, cls_name, "train")
        if not os.path.exists(class_path):
            continue
        
        file_list = [f for f in os.listdir(class_path) if f.endswith('.off')]
        for file_name in tqdm(file_list, desc=f"{cls_name}", leave=False):
            file_path = os.path.join(class_path, file_name)
            try:
                mesh = trimesh.load(file_path)
                for orientation_idx in range(num_orientations):
                    angle = (360 / num_orientations) * orientation_idx
                    rotated_mesh = rotate_mesh(mesh, angle)
                    voxelized = rotated_mesh.voxelized(pitch=rotated_mesh.scale / voxel_dim)
                    voxel_matrix = voxelized.matrix.astype(np.float32)
                    voxel_matrix = zoom(voxel_matrix, (
                        voxel_dim / voxel_matrix.shape[0],
                        voxel_dim / voxel_matrix.shape[1],
                        voxel_dim / voxel_matrix.shape[2]
                    ))
                    X.append(voxel_matrix)
                    y_class.append(label_map[cls_name])
                    y_orientation.append(orientation_idx)
            except Exception as e:
                print(f"error in: {file_path}: {e}")
    
    X = np.array(X)[..., np.newaxis]
    y_class = np.array(y_class)
    y_orientation = np.array(y_orientation)
    return X, y_class, y_orientation, label_map

In [6]:
X, y_class, y_orientation, label_map = load_voxel_dataset_with_orientation(
    dataset_root, voxel_dim=32, selected_classes=selected_classes
)
print("Dataset is ready", X.shape, y_class.shape, y_orientation.shape)

chair:  10%|████████████▊                                                                                                                     | 88/889 [02:22<09:38,  1.39it/s]


chair:  20%|█████████████████████████▊                                                                                                       | 178/889 [05:00<22:10,  1.87s/it]


chair:  30%|██████████████████████████████████████▉                                                                                          | 268/889 [08:03<11:53,  1.15s/it]


chair:  40%|███████████████████████████████████████████████████▉                                                                             | 358/889 [10:51<15:07,  1.71s/it]


chair:  50%|█████████████████████████████████████████████████████████████████                                                                | 448/889 [14:14<17:40,  2.40s/it]


...

toilet:  92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 315/344 [12:25<02:42,  5.60s/it]


dresser:  30%|█████████████████████████████████████▊                                                                                          | 59/200 [03:10<08:12,  3.49s/it]


dresser:  74%|██████████████████████████████████████████████████████████████████████████████████████████████▌                                | 149/200 [09:06<04:43,  5.55s/it]


Classes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [2:58:53<00:00, 1073.32s/it]


Dataset is ready (47892, 32, 32, 32, 1) (47892,) (47892,)


In [7]:
np.save("X_voxel_oriented.npy", X)
np.save("y_class_oriented.npy", y_class)
np.save("y_orientation_oriented.npy", y_orientation)